# **hkl_soleil** E4CV

At X-ray synchrotrons, the vertical *E4CV* {ref}`geometry <geometries-hkl_soleil-e4cv>`
is more common (than *E4CH*, common in labs) due to the polarization of the X-rays.

![E4CV geometry](../_static/3S+1D-fourc.png)

## Setup the *E4CV* diffractometer in **hklpy2**

The *hkl_soleil* *E4CV* [geometry](https://people.debian.org/~picca/hkl/hkl.html)
is described:

axis  | moves    | rotation axis    | vector
---   | ---      | ---              | ---
omega | sample   | {math}`-\vec{y}` | `[0 -1 0]`
chi   | sample   | {math}`\vec{x}`  | `[1 0 0]`
phi   | sample   | {math}`-\vec{y}` | `[0 -1 0]`
tth   | detector | {math}`-\vec{y}` | `[0 -1 0]`

* xrays incident on the $\vec{x}$ direction (1, 0, 0)

## Define _this_ diffractometer

Use the **hklpy2** `creator()` function to create a diffractometer
object.  The diffractometer object will have simulated rotational axes.

We'll provide the geometry and solver names.
By convention, the `name` keyword is the same as the object name.

See the [geometry tables](../diffractometers.rst) for
a more complete description of the available diffractometers.

Create the Python diffractometer object (`fourc`).

In [1]:
import hklpy2

fourc = hklpy2.creator(name="fourc", geometry="E4CV", solver="hkl_soleil")

(example.e4cv.add-sample)=
## Add a sample with a crystal structure

In [2]:
from hklpy2.user import add_sample, calc_UB, cahkl, cahkl_table, pa, set_diffractometer, setor, wh

set_diffractometer(fourc)
add_sample("silicon", a=hklpy2.SI_LATTICE_PARAMETER)

Sample(name='silicon', lattice=Lattice(a=5.431, system='cubic'))

## Setup the UB orientation matrix using *hklpy*

Define the crystal's orientation on the diffractometer using 
the 2-reflection method described by [Busing & Levy, Acta Cryst 22 (1967) 457](https://www.psi.ch/sites/default/files/import/sinq/zebra/PracticalsEN/1967-Busing-Levy-3-4-circle-Acta22.pdf).

### Diffractometer wavelength

Set the diffractometer's X-ray wavelength.  This will be used for both reflections.  `fourc.beam.wavelength` is an
ophyd [Signal](https://blueskyproject.io/ophyd/user/reference/signals.html). Use
its `.put()` method.

In [3]:
fourc.beam.wavelength.put(1.54)

### Specify the first reflection

Provide the set of angles that correspond with the reflection's Miller indices: (_hkl_)

The `setor()` (set orienting reflection) method uses the diffractometer's wavelength *at the time it is called*.  (To add reflections at different wavelengths, add a `wavelength=1.0` keyword argument with the correct value.)  

In [4]:
r1 = setor(4, 0, 0, tth=69.0966, omega=-145.451, chi=0, phi=0)

### Specify the second reflection

In [5]:
r2 = setor(0, 4, 0, tth=69.0966, omega=-145.451, chi=90, phi=0)

### Compute the *UB* orientation matrix

The `calc_UB()` method returns the computed **UB** matrix.

In [6]:
calc_UB(r1, r2)

[[-1.4134285e-05, -1.4134285e-05, -1.156906937382],
 [0.0, -1.156906937469, 1.4134285e-05],
 [-1.156906937469, 1.73e-10, 1.4134285e-05]]

## Report our setup

In [7]:
pa()

diffractometer='fourc'
HklSolver(name='hkl_soleil', version='5.1.2', geometry='E4CV', engine_name='hkl', mode='bissector')
Sample(name='silicon', lattice=Lattice(a=5.431, system='cubic'))
Reflection(name='r_3fc2', h=4, k=0, l=0)
Reflection(name='r_c9e5', h=0, k=4, l=0)
Orienting reflections: ['r_3fc2', 'r_c9e5']
U=[[-1.2217305e-05, -1.2217305e-05, -0.999999999851], [0.0, -0.999999999925, 1.2217305e-05], [-0.999999999925, 1.49e-10, 1.2217305e-05]]
UB=[[-1.4134285e-05, -1.4134285e-05, -1.156906937382], [0.0, -1.156906937469, 1.4134285e-05], [-1.156906937469, 1.73e-10, 1.4134285e-05]]
constraint: -180.0 <= omega <= 180.0
constraint: -180.0 <= chi <= 180.0
constraint: -180.0 <= phi <= 180.0
constraint: -180.0 <= tth <= 180.0
Mode: bissector
beam={'class': 'WavelengthXray', 'source_type': 'Synchrotron X-ray Source', 'energy': 8.050921976530415, 'wavelength': 1.54, 'energy_units': 'keV', 'wavelength_units': 'angstrom'}
h=0, k=0, l=0
omega=0, chi=0, phi=0, tth=0


## Check the orientation matrix

Perform checks with `forward()` ($hkl$ to angle) and
`inverse()` (angle to $hkl$) computations to verify the diffractometer
will move to the same positions where the reflections were identified.

### Constrain the motors to limited ranges

* keep `tth` in the positive range
* keep `omega` in the negative range
* allow for slight roundoff errors
* keep `phi` fixed at zero

First, apply [constraints](../concepts/constraints.rst) to each of the
rotational motors. Constraints are part of the diffractometer's `core`-level
support.

---

**Note**: A constraint does not limit the range of the motor, it *constrains*
the choice of solutions from the `forward()` computation.

---

In [8]:
fourc.core.constraints["tth"].limits = -0.001, 180
fourc.core.constraints["omega"].limits = (-180, 0.001)
fourc.core.constraints

['-180.0 <= omega <= 0.001', '-180.0 <= chi <= 180.0', '-180.0 <= phi <= 180.0', '-0.001 <= tth <= 180.0']

### (400) reflection test

1. Check the `inverse()` (angles -> (_hkl_)) computation.
1. Check the `forward()` ((_hkl_) -> angles) computation.

#### Check `inverse()` at (400)

To calculate the (_hkl_) corresponding to a given set of motor angles,
call `fourc.inverse()`.

The _hkl_ values are provided as a Python [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) structure.

In [9]:
fourc.inverse((-145.451, 0, 0, 69.0966))

Hklpy2DiffractometerPseudoPos(h=6.159871816956, k=0, l=0)

#### Check `forward(400)`

Compute the angles necessary to position the diffractometer
for the given reflection.

---

**Note**:

For the `forward()` computation, more than one set of angles may be used to reach the same crystal reflection.  This test will report the *default* selection.  The *default* selection (which may be changed through methods described in module :mod:`hklpy2.ops`) is the first solution.

function | returns
--- | ---
`cahkl()` | The *default* solution
`cahkl_table()` | Table of all allowed solutions.

---

Before calling `forward()`, make sure we are using the desired operations
*mode*.  `"bissector"` maintains $\omega = (2\theta) / 2$

In [10]:
fourc.core.mode = "bissector"

Here we print the *default* solution (the one returned by calling `cahkl()`.

In [11]:
cahkl(4, 0, 0)

Hklpy2DiffractometerRealPos(omega=-145.450878679139, chi=4.23e-10, phi=0.000700605239, tth=69.098242641722)

---

**Note**: `cahkl()` is a shortcut to `fourc.forward()`.

In [12]:
fourc.forward(4, 0, 0)

Hklpy2DiffractometerRealPos(omega=-145.450879071517, chi=4.23e-10, phi=0.000700049935, tth=69.098241856966)

---

Show the table of *all* `forward()` solutions for $(4\ 0\ 0)$ and $(0\ 4\ 0)$ allowed by the current constraints.  Since this function accepts a *list* of $hkl$  reflections, extra Python syntax is applied.

In [13]:
cahkl_table((4, 0, 0), (0, 4, 0))

======= = ========= ======= ========= =======
(hkl)   # omega     chi     phi       tth    
======= = ========= ======= ========= =======
(4 0 0) 1 -145.4509 0.0     0.0007    69.0982
(4 0 0) 2 -34.5491  180.0   -69.0975  69.0982
(4 0 0) 3 -145.4509 180.0   -179.9993 69.0982
(4 0 0) 4 -34.5491  0.0     -110.9011 69.0982
(0 4 0) 1 -145.4509 90.0007 -89.9994  69.0982
(0 4 0) 2 -145.4509 89.9993 89.9994   69.0982
(0 4 0) 3 -145.4509 90.0007 -90.0006  69.0982
(0 4 0) 4 -145.4509 89.9993 90.0006   69.0982
======= = ========= ======= ========= =======



### (040) reflection test

Repeat the `inverse` and `forward` calculations for the
second orientation reflection.

#### Check `inverse()` at (040)

In [14]:
fourc.inverse(-145.451, 90, 0, 69.0966)

Hklpy2DiffractometerPseudoPos(h=1.7e-10, k=3.999916764179, l=3.876e-09)

#### Check `forward(040)`

In [15]:
fourc.forward(0, 4, 0)

Hklpy2DiffractometerRealPos(omega=-145.450878700115, chi=90.000699948565, phi=-89.972518043795, tth=69.098242599771)

## Scan in reciprocal space using Bluesky

To scan with Bluesky, we need more setup.

In [16]:
from bluesky import RunEngine
from bluesky import SupplementalData
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import databroker

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2
sd = SupplementalData()

RE = RunEngine({})
RE.md = {}
RE.preprocessors.append(sd)
RE.subscribe(cat.v1.insert)
RE.subscribe(bec)

1

Setup the `RE` to save the `fourc` {class}`~hklpy2.blocks.configure.Configuration` with every run.

In [17]:
crw = hklpy2.ConfigurationRunWrapper(fourc)
RE.preprocessors.append(crw.wrapper)

### (_h00_) scan near (400)

In this example, we have no detector.  Still, we add the diffractometer
object in the detector list so that the _hkl_ and motor positions will appear
as columns in the table.

In [18]:
fourc.move(4, 0, 0)
RE(bp.scan([fourc], fourc.h, 3.9, 4.1, 5))



Transient Scan ID: 1     Time: 2025-04-09 22:27:25
Persistent Unique Scan ID: 'a81d4da7-1242-4998-9c2f-a3efb3403b21'
New stream: 'primary'
+-----------+------------+------------+-----------------------+-------------------+------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h | fourc_beam_wavelength | fourc_beam_energy |    fourc_k |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+-----------------------+-------------------+------------+------------+-------------+------------+------------+------------+
|         1 | 22:27:25.3 |      3.900 |                 1.540 |             8.051 |     -0.000 |      0.000 |     -33.569 |      0.000 |   -112.862 |     67.137 |
|         2 | 22:27:25.3 |      3.950 |                 1.540 |             8.051 |      0.000 |      0.000 |     -34.057 |     -0.000 |   -111.884 |     68.115 |
|         3 | 22:27:25.3 |      4.000 |     

('a81d4da7-1242-4998-9c2f-a3efb3403b21',)

### chi scan from (400) to (040)

If we do this with $\omega=-145.4500$ and $2\theta=69.0985$, this will be a scan between the two orientation reflections.

Use `%mov` (IPython *magic* command) to move both motors at the same time.

In [19]:
# same as orientation reflections
RE(bps.mv(fourc.omega,-145.4500, fourc.tth,69.0985))

RE(bp.scan([fourc], fourc.chi, 0, 90, 10))



Transient Scan ID: 2     Time: 2025-04-09 22:27:25
Persistent Unique Scan ID: '8dc6d33f-b5f4-4678-a76a-53cf2e7e2bf2'
New stream: 'primary'
+-----------+------------+------------+-----------------------+-------------------+------------+------------+------------+-------------+------------+------------+
|   seq_num |       time |  fourc_chi | fourc_beam_wavelength | fourc_beam_energy |    fourc_h |    fourc_k |    fourc_l | fourc_omega |  fourc_phi |  fourc_tth |
+-----------+------------+------------+-----------------------+-------------------+------------+------------+------------+-------------+------------+------------+
|         1 | 22:27:25.5 |      0.000 |                 1.540 |             8.051 |     -1.297 |     -0.000 |     -3.784 |    -145.450 |   -108.917 |     69.099 |
|         2 | 22:27:25.5 |     10.000 |                 1.540 |             8.051 |     -1.277 |      0.695 |     -3.727 |    -145.450 |   -108.917 |     69.099 |
|         3 | 22:27:25.5 |     20.000 |     

('8dc6d33f-b5f4-4678-a76a-53cf2e7e2bf2',)

### (_0k0_) scan near (040)

In [20]:
fourc.move(0, 4, 0)
RE(bp.scan([fourc], fourc.k, 3.9, 4.1, 5))



Transient Scan ID: 3     Time: 2025-04-09 22:27:25
Persistent Unique Scan ID: '857245f5-8947-4291-8ac7-06dae6ba929e'
New stream: 'primary'
+-----------+------------+------------+-----------------------+-------------------+------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_h |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+-----------------------+-------------------+------------+------------+-------------+------------+------------+------------+
|         1 | 22:27:25.7 |      3.900 |                 1.540 |             8.051 |      0.000 |      0.000 |    -146.431 |     90.001 |    -90.002 |     67.137 |
|         2 | 22:27:25.7 |      3.950 |                 1.540 |             8.051 |      0.000 |     -0.000 |    -145.943 |     90.001 |    -90.000 |     68.115 |
|         3 | 22:27:25.7 |      4.000 |     

('857245f5-8947-4291-8ac7-06dae6ba929e',)

### (_hk0_) scan near (440)

In [21]:
fourc.move(4, 4, 0)
RE(bp.scan([fourc], fourc.h, 3.9, 4.1, fourc.k, 3.9, 4.1, 5))



Transient Scan ID: 4     Time: 2025-04-09 22:27:25
Persistent Unique Scan ID: 'b837469a-e1b7-45ce-89e1-d2996dfff6c9'
New stream: 'primary'
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|         1 | 22:27:25.9 |      3.900 |      3.900 |                 1.540 |             8.051 |      0.000 |    -128.559 |    135.000 |   -179.999 |    102.882 |
|         2 | 22:27:25.9 |      3.950 |      3.950 |                 1.540 |             8.051 |      0.000 |    -127.628 |    135.000 |   -179.999 |    104.744 |
|         3 | 22:27:26.0 |      4.000 |     

('b837469a-e1b7-45ce-89e1-d2996dfff6c9',)

Move to the (_440_) reflection.

In [22]:
fourc.move((4, 4, 0))
print(f"{fourc.position = }")

fourc.position = Hklpy2DiffractometerPseudoPos(h=4.000000000228, k=3.999999999947, l=1.22e-10)


Repeat the same scan about the (_440_) but use _relative_ positions.

In [23]:
RE(bp.rel_scan([fourc], fourc.h, -0.1, 0.1, fourc.k, -0.1, 0.1, 5))



Transient Scan ID: 5     Time: 2025-04-09 22:27:26
Persistent Unique Scan ID: '6f79f7f5-a07e-444a-a82b-2b37a21ba108'
New stream: 'primary'
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|         1 | 22:27:26.2 |      3.900 |      3.900 |                 1.540 |             8.051 |      0.000 |    -128.559 |    135.000 |   -179.999 |    102.882 |
|         2 | 22:27:26.2 |      3.950 |      3.950 |                 1.540 |             8.051 |      0.000 |    -127.628 |    135.000 |   -179.999 |    104.744 |
|         3 | 22:27:26.2 |      4.000 |     

('6f79f7f5-a07e-444a-a82b-2b37a21ba108',)

### scan $(h40)$ with $\varphi=0$

In [24]:
fourc.core.mode = "constant_phi"
fourc.phi.move(0, wait=True)
wh()

wavelength=1.54
h=-4.0, k=4.0, l=0
omega=-126.6767, chi=135.0, phi=0, tth=106.6465


In [25]:
RE(bp.rel_scan([fourc], fourc.h, -0.1, 0.1, fourc.k, -0.1, 0.1, 5))



Transient Scan ID: 6     Time: 2025-04-09 22:27:26
Persistent Unique Scan ID: '3e568641-b5bd-45b8-8f17-12f435f4186d'
New stream: 'primary'
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|         1 | 22:27:26.4 |     -4.100 |      3.900 |                 1.540 |             8.051 |      0.000 |    -126.653 |    136.432 |          0 |    106.695 |
|         2 | 22:27:26.4 |     -4.050 |      3.950 |                 1.540 |             8.051 |      0.000 |    -126.671 |    135.716 |          0 |    106.659 |
|         3 | 22:27:26.4 |     -4.000 |     

('3e568641-b5bd-45b8-8f17-12f435f4186d',)

### Show the configuration

Print the diffractometer configuration that was saved with the run.

In [26]:
cat.v2[-1].metadata["start"]["diffractometers"]["fourc"]

{'_header': {'datetime': '2025-04-09 22:27:26.396767',
  'hklpy2_version': '0.0.29.dev43+gdc399a0.d20250410',
  'python_class': 'Hklpy2Diffractometer'},
 'name': 'fourc',
 'axes': {'pseudo_axes': ['h', 'k', 'l'],
  'real_axes': ['omega', 'chi', 'phi', 'tth'],
  'axes_xref': {'h': 'h',
   'k': 'k',
   'l': 'l',
   'omega': 'omega',
   'chi': 'chi',
   'phi': 'phi',
   'tth': 'tth'},
  'extra_axes': {'h2': 0, 'k2': 0, 'l2': 0, 'psi': 0}},
 'sample_name': 'silicon',
 'samples': {'sample': {'name': 'sample',
   'lattice': {'a': 1,
    'b': 1,
    'c': 1,
    'alpha': 90.0,
    'beta': 90.0,
    'gamma': 90.0},
   'reflections': {},
   'reflections_order': [],
   'U': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'UB': [[6.283185307179586, 0.0, 0.0],
    [0.0, 6.283185307179586, 0.0],
    [0.0, 0.0, 6.283185307179586]],
   'digits': 4},
  'silicon': {'name': 'silicon',
   'lattice': {'a': 5.431020511,
    'b': 5.431020511,
    'c': 5.431020511,
    'alpha': 90,
    'beta': 90,
    'gamma': 90},
   